<a href="https://colab.research.google.com/github/Lrizika/DS-Unit-1-Sprint-1-Dealing-With-Data/blob/master/module3-databackedassertions/LS_DS_113_Making_Data_backed_Assertions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science - Making Data-backed Assertions

This is, for many, the main point of data science - to create and support reasoned arguments based on evidence. It's not a topic to master in a day, but it is worth some focused time thinking about and structuring your approach to it.

## Lecture - generating a confounding variable

The prewatch material told a story about a hypothetical health condition where both the drug usage and overall health outcome were related to gender - thus making gender a confounding variable, obfuscating the possible relationship between the drug and the outcome.

Let's use Python to generate data that actually behaves in this fashion!

In [35]:
import random
dir(random)  # Reminding ourselves what we can do here

['BPF',
 'LOG4',
 'NV_MAGICCONST',
 'RECIP_BPF',
 'Random',
 'SG_MAGICCONST',
 'SystemRandom',
 'TWOPI',
 '_BuiltinMethodType',
 '_MethodType',
 '_Sequence',
 '_Set',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_acos',
 '_bisect',
 '_ceil',
 '_cos',
 '_e',
 '_exp',
 '_inst',
 '_itertools',
 '_log',
 '_pi',
 '_random',
 '_sha512',
 '_sin',
 '_sqrt',
 '_test',
 '_test_generator',
 '_urandom',
 '_warn',
 'betavariate',
 'choice',
 'choices',
 'expovariate',
 'gammavariate',
 'gauss',
 'getrandbits',
 'getstate',
 'lognormvariate',
 'normalvariate',
 'paretovariate',
 'randint',
 'random',
 'randrange',
 'sample',
 'seed',
 'setstate',
 'shuffle',
 'triangular',
 'uniform',
 'vonmisesvariate',
 'weibullvariate']

In [36]:
# Let's think of another scenario:
# We work for a company that sells accessories for mobile phones.
# They have an ecommerce site, and we are supposed to analyze logs
# to determine what sort of usage is related to purchases, and thus guide
# website development to encourage higher conversion.

# The hypothesis - users who spend longer on the site tend
# to spend more. Seems reasonable, no?

# But there's a confounding variable! If they're on a phone, they:
# a) Spend less time on the site, but
# b) Are more likely to be interested in the actual products!

# Let's use namedtuple to represent our data

from collections import namedtuple
# purchased and mobile are bools, time_on_site in seconds
User = namedtuple('User', ['purchased','time_on_site', 'mobile'])

example_user = User(False, 12, False)
print(example_user)

User(purchased=False, time_on_site=12, mobile=False)


In [37]:
# And now let's generate 1000 example users
# 750 mobile, 250 not (i.e. desktop)
# A desktop user has a base conversion likelihood of 10%
# And it goes up by 1% for each 15 seconds they spend on the site
# And they spend anywhere from 10 seconds to 10 minutes on the site (uniform)
# Mobile users spend on average half as much time on the site as desktop
# But have three times as much base likelihood of buying something

random.seed(42)
users = []

for _ in range(25000):
  # Desktop users
  time_on_site = random.uniform(10, 600)
  purchased = random.random() < 0.1 + (time_on_site / 1500)
  users.append(User(purchased, time_on_site, False))
  
for _ in range(75000):
  # Mobile users
  time_on_site = random.uniform(5, 300)
  purchased = random.random() < 0.3 + (time_on_site / 1500)
  users.append(User(purchased, time_on_site, True))
  
random.shuffle(users)
print(users[:10])

[User(purchased=False, time_on_site=208.07685224635028, mobile=True), User(purchased=False, time_on_site=220.13450387417356, mobile=True), User(purchased=False, time_on_site=48.43029711411871, mobile=True), User(purchased=False, time_on_site=209.99018515100295, mobile=False), User(purchased=True, time_on_site=512.6387770989866, mobile=False), User(purchased=False, time_on_site=47.99023172657714, mobile=True), User(purchased=True, time_on_site=279.18188031647065, mobile=True), User(purchased=False, time_on_site=191.53652796743236, mobile=True), User(purchased=False, time_on_site=175.5023757905157, mobile=True), User(purchased=False, time_on_site=75.00162575347531, mobile=True)]


In [38]:
# Let's put this in a dataframe so we can look at it more easily
import pandas as pd
user_data = pd.DataFrame(users)
user_data.head()

,purchased,time_on_site,mobile
0,False,208.076852,True
1,False,220.134504,True
2,False,48.430297,True
3,False,209.990185,False
4,True,512.638777,False


In [39]:
# Let's use crosstabulation to try to see what's going on
pd.crosstab(user_data['purchased'], user_data['time_on_site'])

time_on_site,5.001889,5.009356,5.012074,5.012951,5.015756,5.029236,5.031183,5.036888,5.044437,5.046545,5.050565,5.058310,5.058928,5.067053,5.073703,5.073877,5.074111,5.076228,5.078195,5.078407,5.078464,5.092007,5.094205,5.094860,5.113765,5.116852,5.121920,5.121937,5.122144,5.127879,5.129848,5.133064,5.134749,5.139325,5.140319,5.153952,5.155801,5.156736,5.158816,5.161282,...,598.761067,598.782038,598.795876,598.817466,598.823189,598.830099,598.843328,598.905537,598.908853,599.081660,599.088120,599.094461,599.128501,599.141428,599.152746,599.161602,599.239236,599.249623,599.254755,599.256484,599.296249,599.349683,599.356831,599.366288,599.432812,599.485089,599.492884,599.570672,599.576656,599.580660,599.610125,599.639266,599.668946,599.672630,599.730099,599.747484,599.761076,599.889748,599.940736,599.945619
purchased,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
False,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0,1,1,1,1,1,1,0,0,1,1,1,1,1,0,1,0,1,1,0,1,1,1,...,1,0,1,1,1,1,0,1,0,0,1,0,0,0,0,1,1,0,1,0,0,1,1,0,1,1,1,0,1,0,0,1,0,1,1,0,1,1,1,0
True,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,...,0,1,0,0,0,0,1,0,1,1,0,1,1,1,1,0,0,1,0,1,1,0,0,1,0,0,0,1,0,1,1,0,1,0,0,1,0,0,0,1


In [40]:
# OK, that's not quite what we want
# Time is continuous! We need to put it in discrete buckets
# Pandas calls these bins, and pandas.cut helps make them
!pip3 install pandas==0.25.1
time_bins = pd.cut(user_data['time_on_site'], 6)  # 5 equal-sized bins
pd.crosstab(user_data['purchased'], time_bins)

time_on_site,"(4.407, 104.159]","(104.159, 203.316]","(203.316, 302.474]","(302.474, 401.631]","(401.631, 500.788]","(500.788, 599.946]"
purchased,,,,,,
False,20302,18459,16244,2772,2567,2163
True,8968,10923,12568,1359,1742,1933


In [41]:
# We can make this a bit clearer by normalizing (getting %)
pd.crosstab(user_data['purchased'], time_bins, normalize='columns')

time_on_site,"(4.407, 104.159]","(104.159, 203.316]","(203.316, 302.474]","(302.474, 401.631]","(401.631, 500.788]","(500.788, 599.946]"
purchased,,,,,,
False,0.693611,0.628242,0.563793,0.671024,0.59573,0.528076
True,0.306389,0.371758,0.436207,0.328976,0.40427,0.471924


In [42]:
# That seems counter to our hypothesis
# More time on the site can actually have fewer purchases

# But we know why, since we generated the data!
# Let's look at mobile and purchased
pd.crosstab(user_data['purchased'], user_data['mobile'], normalize='columns')

mobile,False,True
purchased,,
False,0.70004,0.60008
True,0.29996,0.39992


In [43]:
# Yep, mobile users are more likely to buy things
# But we're still not seeing the *whole* story until we look at all 3 at once

# Live/stretch goal - how can we do that?

pd.crosstab(user_data['purchased'], [time_bins, user_data['mobile']], normalize='columns')


time_on_site (4.407, 104.159]            ... (401.631, 500.788] (500.788, 599.946]
mobile                  False     True   ...              False              False
purchased                                ...                                      
False                0.861831  0.666574  ...            0.59573           0.528076
True                 0.138169  0.333426  ...            0.40427           0.471924

[2 rows x 9 columns]